#LOAD IMAGES FROM MNIST DATABASE

In [1]:
!pip install -r https://raw.githubusercontent.com/Lasagne/Lasagne/master/requirements.txt

    100% |████████████████████████████████| 2.9MB 5.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/e4/33/9d/880c86cc0db0b29d34fb6c62256322a4ea1694404192c2b408
Successfully built Theano
pymc3 3.6 has requirement theano>=1.0.0, but you'll have theano 0.8.2 which is incompatible.
  Found existing installation: Theano 1.0.4
    Uninstalling Theano-1.0.4:
      Successfully uninstalled Theano-1.0.4


In [2]:
!pip install https://github.com/Lasagne/Lasagne/archive/master.zip


     \ 1.2MB 134.1MB/s
  Stored in directory: /tmp/pip-ephem-wheel-cache-vqqdvhzy/wheels/ca/4a/00/87f1777b229481fe76562df7c0cfb993bc88ed0cc37e3f0ed4
Successfully built Lasagne


In [0]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 19 22:57:30 2019

@author: Sumedh Nakod
"""
import numpy as np
import os

#downloading files from yann lecun website
def load_dataset():
    def save(filename, source='http://yann.lecun.com/exdb/mnist/'):
        print("downloading", filename)
        import urllib.request
        urllib.request.urlretrieve(source+filename,filename)
        
#unzipping images
    import gzip
        
    def load_mnist_images(filename):
        if not os.path.exists(filename):
            save(filename)
        with gzip.open(filename,'rb') as f:
            data=np.frombuffer(f.read(),np.uint8,offset=16)
            

#coversion into Array of images                
            data=data.reshape(-1,1,28,28) #each image is of pixel 28x28
            return data/np.float32(256)
            #byte to float
        
    def load_mnist_lables(filename):
        if not os.path.exists(filename):
            save(filename)
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(),np.uint8,offset=8)
        return data
        
    X_train = load_mnist_images('train-images-idx3-ubyte.gz')
    y_train = load_mnist_lables('train-labels-idx1-ubyte.gz')
    X_test = load_mnist_images('t10k-images-idx3-ubyte.gz')
    y_test = load_mnist_lables('t10k-labels-idx1-ubyte.gz')
        
    return X_train,y_train,X_test,y_test

In [17]:
X_train,y_train,X_test,y_test = load_dataset()

downloading train-images-idx3-ubyte.gz
downloading train-labels-idx1-ubyte.gz
downloading t10k-images-idx3-ubyte.gz
downloading t10k-labels-idx1-ubyte.gz


In [0]:
import lasagne
import theano
import theano.tensor as T
def build_NN(input_var=None):
  l_in=lasagne.layers.InputLayer(shape=(None,1,28,28),input_var=input_var)
  #drop 20% edges to prevent overfitting
  l_in_drop=lasagne.layers.DropoutLayer(l_in,p=0.2)
  #hidden layers
  l_hid1=lasagne.layers.DenseLayer(l_in_drop,num_units=800,
                                    nonlinearity=lasagne.nonlinearities.rectify,
                                    W=lasagne.init.GlorotUniform())#initializes weights randomly
  #drop 50% from hidden layer
  l_hid1_drop=lasagne.layers.DropoutLayer(l_hid1,p=0.5)
  #hidden layers
  l_hid2 =lasagne.layers.DenseLayer(l_hid1_drop,num_units=800,
                                    nonlinearity=lasagne.nonlinearities.rectify,
                                    W=lasagne.init.GlorotUniform())
  #drop 50% from hidden layer 2
  l_hid2_drop=lasagne.layers.DropoutLayer(l_hid2,p=0.5)
  
  #Output Layer
  l_out = lasagne.layers.DenseLayer(l_hid2_drop,num_units=10,
                                    nonlinearity=lasagne.nonlinearities.softmax)#specifies each output is between 0-1 and max of those will be the predicted digit
  return l_out

#training network
input_var=T.tensor4('inputs') #empty 4D array
target_var=T.ivector('targets') #an empty 1D int array

network = build_NN(input_var)

#error function
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction,target_var)#difference between original and predicted value
loss = loss.mean()

params = lasagne.layers.get_all_params(network) #current weights
updates = lasagne.updates.nesterov_momentum(loss,params,
                                            learning_rate=0.01,
                                            momentum = 0.9)

train_fn= theano.function([input_var , target_var],loss,updates=updates)

  
  

In [27]:
num_training_steps = 20

for steps in range(num_training_steps):
  train_err = train_fn(X_train,y_train)
  print("Current step is"+ str(steps))

Current step is0
Current step is1
Current step is2
Current step is3
Current step is4
Current step is5
Current step is6
Current step is7
Current step is8
Current step is9
Current step is10
Current step is11
Current step is12
Current step is13
Current step is14
Current step is15
Current step is16
Current step is17
Current step is18
Current step is19


In [22]:

test_prediction = lasagne.layers.get_output(network)
val_fn = theano.function([input_var],test_prediction)

val_fn([X_test[0]])

array([[0.10868358, 0.10641307, 0.0788198 , 0.15119967, 0.07809951,
        0.12790207, 0.08488368, 0.13312842, 0.05080178, 0.08006842]])

In [24]:
y_test[0]


7

In [26]:
#test data set
test_prediction = lasagne.layers.get_output(network,deterministic = True)
test_acc = T.mean(T.eq(T.argmax(test_prediction , axis=1),target_var),dtype=theano.config.floatX)

acc_fn = theano.function([input_var,target_var],test_acc)
acc_fn(X_test,y_test)

array(0.2624)